In [2]:
import json
from kafka import KafkaConsumer
from river import (
    compose, 
    linear_model, 
    preprocessing, 
    metrics, 
    anomaly
)
import datetime
import pickle
import os
import pandas as pd

In [3]:
# Configuration
KAFKA_TOPIC = 'transactions'
KAFKA_BROKERS = 'kafka-producer:29092'  # Adjust as needed
MODEL_PATH = 'river_model.pkl'
DATA_PATH = 'river_data.pkl'

In [4]:
#Data processing functions
def extract_device_info(x):
    x_ = x['device_info']
    return {
        'os': x_['os'],
        'browser': x_['browser'],
    }

In [5]:
def load_or_create_model():
    """Load existing model or create a new one"""
    # Create a new model pipeline
    pipe1 = compose.Select(
        "amount",
        "account_age_days",
        "cvv_provided",
        "billing_address_match"
    )
    pipe2 = compose.Select(
        "currency",
        "merchant_id",
        "payment_method",
        "product_category",
        "transaction_type",
        "user_agent"
    )
    pipe2 |= preprocessing.OrdinalEncoder()
    pipe3 = compose.Select(
        "device_info"
    )
    pipe3 |= compose.FuncTransformer(
        extract_device_info,
    )
    pipe3 |= preprocessing.OrdinalEncoder()
    pipe = pipe1 + pipe2 + pipe3
    model = pipe | linear_model.LogisticRegression()
    #Save the model to future use
    with open(MODEL_PATH, 'wb') as f:
        pickle.dump(model, f)
    return model

In [6]:
model = load_or_create_model()

In [7]:
model[-1].weights

{}

In [12]:
model[0]

TransformerUnion (
  Select (
    account_age_days
    amount
    billing_address_match
    cvv_provided
  ),
  Pipeline (
    Select (
      currency
      merchant_id
      payment_method
      product_category
      transaction_type
      user_agent
    ),
    OrdinalEncoder (
      unknown_value=0
      none_value=-1
    )
  ),
  Pipeline (
    Select (
      device_info
    ),
    FuncTransformer (
      func="extract_device_info"
    ),
    OrdinalEncoder (
      unknown_value=0
      none_value=-1
    )
  )
)

In [28]:
from river import ensemble

dir(ensemble)

['ADWINBaggingClassifier',
 'ADWINBoostingClassifier',
 'AdaBoostClassifier',
 'BOLEClassifier',
 'BaggingClassifier',
 'BaggingRegressor',
 'EWARegressor',
 'LeveragingBaggingClassifier',
 'SRPClassifier',
 'SRPRegressor',
 'StackingClassifier',
 'VotingClassifier',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'annotations',
 'bagging',
 'boosting',
 'ewa',
 'stacking',
 'streaming_random_patches',
 'voting']

In [30]:
dir(river.tree.splitter)

['DynamicQuantizer',
 'EBSTSplitter',
 'ExhaustiveSplitter',
 'GaussianSplitter',
 'HistogramSplitter',
 'QOSplitter',
 'Quantizer',
 'Splitter',
 'StaticQuantizer',
 'TEBSTSplitter',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'annotations',
 'base',
 'ebst_splitter',
 'exhaustive_splitter',
 'gaussian_splitter',
 'histogram_splitter',
 'nominal_splitter_classif',
 'nominal_splitter_reg',
 'qo_splitter',
 'sgt_quantizer',
 'tebst_splitter']